In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
import logging
import datetime
import sys
import dask.dataframe as dd

In [ ]:
!mkdir -p tmp

In [ ]:
#export
def test_eq(a,b): assert a==b, f'{a}, {b}'

In [ ]:
#export
def configure_logging(log_dir, log_name, log_lvl='DEBUG', con_log_lvl='INFO'):
    log = logging.getLogger('root')
    already_initialized = any(filter(lambda h: isinstance(h, logging.StreamHandler), log.handlers))
    if already_initialized:
        print("Logging already initialized")
        return logging.getLogger('root')

    numeric_level = getattr(logging, log_lvl, None)
    log_format = '%(levelname)5s [%(asctime)s] %(name)s: %(message)s'
    date_format = '%Y-%m-%d %H:%M:%S'
    logging.basicConfig(
        filename=f'{log_dir}/{log_name}_{datetime.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")}.txt',
        level=numeric_level,
        format=log_format,
        datefmt=date_format)
    log = logging.getLogger('root')
    ch = logging.StreamHandler()
    ch.setLevel(getattr(logging, con_log_lvl, None))
    ch.setFormatter(logging.Formatter(log_format, date_format))
    log.addHandler(ch)

    return log

In [ ]:
#export
def setup_dataframe_copy_logging(log):
    if not '_original_copy' in dir(pd.DataFrame):
        log.debug('Patching up DataFrame.copy')
        pd.DataFrame._original_copy = pd.DataFrame.copy
    else:
        log.debug('Patching up DataFrame.copy :: already done - skipping.')

    def _loud_copy(self, deep=True):
        log.debug(f'Copying {sys.getsizeof(self) / 1024 / 1024:.1f} MiB (deep={deep})')
        return pd.DataFrame._original_copy(self, deep)

    pd.DataFrame.copy = _loud_copy

In [ ]:
log = configure_logging('./tmp', 'test_log', con_log_lvl='DEBUG')
setup_dataframe_copy_logging(log)

In [ ]:
df = pd.DataFrame({'a':[1,2,3]})
df2 = df.copy()

In [ ]:
#export
n_total_series = 30490
n_days_total = 1913
raw_dir = 'raw'

In [ ]:
#export
def read_series_sample(log, n):
    df = dd.read_csv(
        f'{raw_dir}/sales_train_validation.csv'
    ).sample(frac = n / n_total_series)
    log.debug(f"Read {len(df)} series")
    return df

In [ ]:
sample = read_series_sample(log, 13)
test_eq(13, len(sample))

In [ ]:
#export
def melt_sales_series(df_sales_train):
    id_columns = [col for col in df_sales_train.columns if 'id' in col]
    sales_columns = [col for col in df_sales_train.columns if 'd_' in col]
    cat_columns = [col for col in id_columns if col != 'id']

    df_sales_train_melt = df_sales_train.melt(
        id_vars=id_columns,
        var_name='day_id',
        value_name='sales'
    )

    df_sales_train_melt['sales'] = df_sales_train_melt['sales'].astype('int16')

    return df_sales_train_melt

In [ ]:
sample_melt = melt_sales_series(sample)

In [ ]:
test_eq(n_days_total * 13, len(sample_melt))

In [ ]:
#export
def extract_day_ids(df_sales_train_melt):
    sales_columns = [f'd_{col}' for col in range(1, n_days_total+1)]
    mapping = {col: int(col.split('_')[1]) for col in sales_columns}
    df_sales_train_melt['day_id'] = df_sales_train_melt['day_id'].map(mapping)

    import datetime
    d_1_date = pd.to_datetime('2011-01-29')
    mapping = {day:(d_1_date + datetime.timedelta(days=day-1)) for day in range(1, n_days_total+1)}
    df_sales_train_melt['day_date'] = df_sales_train_melt['day_id'].map(mapping)

    mapping = {day:str((d_1_date + datetime.timedelta(days=day-1)).date()) for day in range(1, n_days_total+1)}
    # gonna need it for joining with calendars & stuff
    df_sales_train_melt['day_date_str'] = df_sales_train_melt['day_id'].map(mapping)

    df_sales_train_melt['day_date_str'] = df_sales_train_melt['day_date_str'].astype('category')
    df_sales_train_melt['day_id'] = df_sales_train_melt['day_id'].astype('int16')
    df_sales_train_melt['month_id'] = df_sales_train_melt['day_date'].dt.month.astype('uint8')

    return df_sales_train_melt

In [ ]:
sample_melt = extract_day_ids(sample_melt)